## Expeciation-Maximization algorithm for Gaussian Mixture model

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import time

#define number of clusters to generate data
n_clusters = 3

def generate_data(mean, cov, n_clusters, n_samples):
    '''Generates a mixture of bivariate Gassian distributions'''
    X = []
    for i in range(n_clusters):
        X.append(np.random.multivariate_normal(mean[i], cov[i], n_samples[i]))    
    X = np.vstack(X)
    return X


n_clusters = 3
mean = [[11, 19], [15, 15], [19, 19]]
cov = [[[1, 0],
        [0, 1]],
       [[3, 0],
        [0, 3]],
        [[1, 0],
        [0, 1]]]
n_samples = [500, 1200, 500]
X = generate_data(mean, cov, n_clusters, n_samples)

def initialize_clusters(X):
    '''Generates initial clusters parameters'''
    idx = np.random.randint(low=0, high=len(X), size=n_clusters)
    mu = np.array([X[index] for index in idx])
    G = [[[1,0], [0,1]] for i in range(n_clusters)]
    pi = [1/n_clusters]*n_clusters
    return np.array(mu), G, pi

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, push_notebook, show
from bokeh.palettes import all_palettes
from bokeh.models import ColumnDataSource
from bokeh.models import LinearColorMapper
from bokeh.palettes import Viridis3, Viridis256
output_notebook()

class GMMPlot:
    
    def __init__(self, dummy=None):
        self.dummy = dummy
    
    def _get_elipses(self, G):
        s00 = [g[0,0]*2 for g in G]
        s11 = [g[1,1]*2 for g in G]
        v = [np.linalg.eig(g)[1] for g in G]
        angle = [np.arctan(vi[0,0]/vi[1,0]) if vi[1,0] != 0 else 0.5*np.pi for vi in v]
        return s00, s11, angle
    
    def create(self, X, mu, w, G, cost, i):
        cl = np.array([0,1,2])
        Xw = np.column_stack((X, w.dot(cl)))
        self.Xw = Xw
        self.source_xyc  = ColumnDataSource(pd.DataFrame(Xw, columns=['x', 'y', 'w']))
        self.plot = figure()
        self.plot.title.text = "Iteration #%d" %(i)
        self.plot.title.text_font_size = '14pt'
        self.plot.title.align = 'center'
        colors = ['royalblue', 'green', 'orange']
        mapper = LinearColorMapper(palette=Viridis256, low=0, high=2)
        self.plot.scatter(x='x', y='y',
                             fill_color={'field': 'w', 'transform': mapper},
                             source=self.source_xyc,
                             line_color=None)
        self.source_mu = ColumnDataSource(pd.DataFrame(mu, columns=['x', 'y']))
        self.plot.circle(x='x', y='y',
                            source=self.source_mu, fill_color='salmon',
                            line_color='red',
                            size=10,
                            line_width=1,
                            level='overlay')
        s00, s11, angle = self._get_elipses(G)
        self.source_E = ColumnDataSource(dict(x=mu[:, 0], y=mu[:,1], w=s00, h=s11, angle=angle))
        self.plot.ellipse(x="x", y="y", width="w",
                            height="h", angle='angle',
                            source=self.source_E, fill_color=None,
                            line_color='red',
                            line_width=2)
        self.handle = show(self.plot, notebook_handle=True)
        
    def update(self, X, mu, w, G, cost, i):
        self.plot.title.text = "Iteration #%d" %(i)
        s00, s11, angle = self._get_elipses(G)
        self.source_E.data = {'x' : mu[:, 0], 'y' : mu[:,1], 'w' : s00, 'h' : s11, 'angle' : angle}
        self.source_mu.data = {'x' : mu[:,0], 'y' : mu[:,1]}
        cl = np.array([0,1,2])
        Xw = np.column_stack((X, w.dot(cl)))
        self.source_xyc.data  = {'x' : Xw[:, 0], 'y' : Xw[:, 1], 'w' : Xw[:, 2]}
        self.Xw = Xw
        push_notebook(handle=self.handle)

Loading BokehJS ...

In [4]:
def EMGMM(X):
    w = np.zeros((len(X), n_clusters))
    n = np.zeros(n_clusters)
    mu, G, pi = initialize_clusters(X)
    bh_plot = GMMPlot()
    bh_plot.create(X, mu, w, np.array(G), cost=0, i=0)
    
    for i in range(31):
        for xi in range(X.shape[0]):
            for k in range(n_clusters):
                w[xi, k] = pi[k] * sp.stats.multivariate_normal.pdf(X[xi,:], mean=mu[k], cov=G[k])
            if np.sum(w[xi, :]) != 0:
                w[xi, :] = w[xi, :] / np.sum(w[xi, :])
        
        for k in range(n_clusters):
            n[k] = np.sum(w[:, k], axis=0)
            mu[k] = 1./n[k] * np.dot(w[:, k], X)
            pi[k] = n[k]/len(X)
            G[k] = np.zeros((X.shape[1], X.shape[1]))
            for ni in range(X.shape[0]):
                G[k] += w[ni,k] * np.outer((X[ni, :] - mu[k]).T , (X[ni, :] - mu[k]))
            G[k] = 1./n[k] * G[k]
        bh_plot.update(X, mu, w, np.array(G), cost=0, i=i)
    return w, G, mu
w, G, mu = EMGMM(X)

/home/gabriel/anaconda3/lib/python3.6/site-packages/bokeh/core/json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):
